In [48]:
# Importa bibliotecas essenciais para o notebook
# - pathlib: manipulação de caminhos de arquivos
# - pandas: manipulação de dados
# - scipy.stats: funções estatísticas como z-score
from pathlib import Path

import pandas as pd
from scipy import stats

In [49]:
# Configura o pandas para exibir números float com 8 casas decimais
# e usando '_' como separador de milhar (ex: 0_12345678)
pd.options.display.float_format = "{:_.8f}".format

In [50]:
# Define os diretórios de dados:
# - DADOS: onde estão os arquivos de votos completos
# - AMOSTRAS: onde serão salvos os arquivos de amostras extraídas
DADOS: Path = Path(".").parent.resolve() / "data" / "raw"
AMOSTRAS: Path = Path(".").parent.resolve() / "data" / "sample"

In [51]:
# Define o nível de significância (alpha = 5%) e calcula o valor z
# correspondente à normal padrão para um IC bicaudal (95%)
alpha = .05
z = stats.norm.ppf(1 - alpha / 2)

# Carga dos dados

In [52]:
# Cria um dicionário para padronizar os nomes dos candidatos
# Isso ajuda a consolidar votos de diferentes grafias ou variações
names_map = {
    "Luiz Inácio Lula Da Silva": "Lula",
    "Jose Maria Eymael": "Eymael",
    "Constituinte Eymael": "Eymael",
    "Kelmon Luis Da Silva Souza": "Padre Kelmon",
    "Brancos": "Branco",
    "Voto Branco": "Branco",
    "Luiz Felipe Chaves D Avila": "Felipe D'Avila",
    "Soraia Thronicke": "Soraya Thronicke",
    "Soraya Vieira Thronicke": "Soraya Thronicke",
    "Ciro Ferreira Gomes": "Ciro Gomes",
    "Sofia Padua Manzano": "Sofia Manzano",
    "Jair Messias Bolsonaro": "Jair Bolsonaro",
    "Simone Nassar Tebet": "Simone Tebet",
    "Leonardo Péricles Vieira Roque": "Léo Péricles",
    "Voto Nulo": "Nulo",
    "Nulos": "Nulo",
    "Vera Lucia Pereira Da Silva Salgado": "Vera Lucia",
    "Vera": "Vera Lucia",
    "Felipe D´Avila": "Felipe D'Avila",
}

In [ ]:
# Carrega os dados populacionais
dfs = []

for file in DADOS.glob("*.csv"):
    cargo, turno, *_ = file.stem.split("_")
    df = (
        pd.read_csv(file, dtype="category")
        .iloc[:, -2:]
        .rename(columns=str.casefold)
        .rename(columns=lambda name: "nome" if name != "estado" else name)
        .assign(
            nome=lambda df_: df_["nome"].str.title(),
            cargo=cargo,
            turno=int(turno)
        )
        .assign(
            nome=lambda df_: df_["nome"].apply(lambda s: names_map.get(s, s))
        )
    )
    dfs.append(df)

populacao = pd.concat(dfs).astype({
    "estado": "category", "cargo": "category"
})

# Geração das amostras

In [54]:
sample_sizes = (
    pd.DataFrame([
        ['AC', 1, "Governador", 455_438, 2389],
        ['AC', 1, "Senador", 455_438, 2389],
        ['RO', 1, "Governador", 925_763, 2395],
        ['RO', 1, "Senador", 925_763, 2395],
        ['RO', 2, "Governador", 925563, 2395],
        ["AC", 1, "Presidente", 455_903, 110],
        ["RO", 1, "Presidente", 926_827, 224],
        ["AM", 1, "Presidente", 2_113_771, 512],
        ["RR", 1, "Presidente", 305_404, 74],
        ["AP", 1, "Presidente", 442_842, 107],
        ["PA", 1, "Presidente", 4_789_311, 1_159],
        ["TO", 1, "Presidente", 891_449, 216],
        ["AC", 2, "Presidente", 420_760, 104],
        ["RO", 2, "Presidente", 926_517, 230],
        ["AM", 2, "Presidente", 2_067_875, 513],
        ["RR", 2, "Presidente", 286_269, 71],
        ["AP", 2, "Presidente", 400_683, 99],
        ["PA", 2, "Presidente", 4_701_740, 1_167],
        ["TO", 2, "Presidente", 871_238, 216],

    ],
    columns=["estado", "turno", "cargo", "N", "n"])
    .set_index(["cargo", "estado", "turno"])
)
sample_sizes

N     n
cargo      estado turno               
Governador AC     1       455438  2389
Senador    AC     1       455438  2389
Governador RO     1       925763  2395
Senador    RO     1       925763  2395
Governador RO     2       925563  2395
Presidente AC     1       455903   110
           RO     1       926827   224
           AM     1      2113771   512
           RR     1       305404    74
           AP     1       442842   107
           PA     1      4789311  1159
           TO     1       891449   216
           AC     2       420760   104
           RO     2       926517   230
           AM     2      2067875   513
           RR     2       286269    71
           AP     2       400683    99
           PA     2      4701740  1167
           TO     2       871238   216

In [ ]:
# Cria as amomstras
random_state = 789

for (uf, turno, cargo), tamanho in sample_sizes.iterrows():
    sample = (
        populacao.query(
            f"estado == {uf!r} and turno == {turno} and cargo == {cargo!r}"
        )
        .sort_values(by="nome")
        .sample(tamanho.n, replace=False, random_state=random_state)
    )
    sample[["nome", "estado"]].to_csv(
        AMOSTRAS / f"{cargo}_{turno}_turno_{uf}.csv",
    )
    print(uf, turno, cargo, sample.shape[0])

In [ ]:
# Carrega os dados das amostras criadas
dfs = []

for file in AMOSTRAS.glob("*.csv"):
    cargo, turno, *_ = file.stem.split("_")
    df = (
        pd.read_csv(file, dtype="category")
        .iloc[:, -2:]
        .rename(columns=str.casefold)
        .rename(columns=lambda name: "nome" if name != "estado" else name)
        .assign(
            nome=lambda df_: df_["nome"].str.title(),
            cargo=cargo,
            turno=int(turno)
        )
        .assign(
            nome=lambda df_: df_["nome"].apply(lambda s: names_map.get(s, s))
        )
    )
    dfs.append(df)

amostra = (
    pd.concat(dfs)
    .astype({
        "estado": "category", "cargo": "category"
    })
)

# Proporções

## Amostrais

### Governador e Senador

In [57]:
proporcoes_aass = (
    amostra
    .query("cargo != 'Presidente'")
    .groupby(["cargo", "estado", "turno"])
    ["nome"].apply(lambda s: s.value_counts(normalize=True))
    .to_frame("p")
    .assign(nome=lambda df_: df_.index.get_level_values(3))
    .droplevel(3)
    .join(sample_sizes)
    .assign(
        f=lambda df_: df_["n"] / df_["N"],
        q=lambda df_: 1 - df_["p"],
        var_p=lambda df_: (1 - df_["f"]) * df_["p"] * df_["q"] / (df_["n"] - 1),
        ep_p =lambda df_: df_["var_p"].pow(.5),
        low_ci=lambda df_: df_["p"] - z * df_["ep_p"],
        upr_ci=lambda df_: df_["p"] + z * df_["ep_p"],
    )
)
proporcoes_aass

/var/folders/xs/y768pf612tscp94by_62zm040000gn/T/ipykernel_26301/1423493882.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["cargo", "estado", "turno"])


p                  nome       N     n  \
cargo      estado turno                                                  
Governador AC     1     0.53034743        Gladson Cameli  455438  2389   
                  1     0.22645458           Jorge Viana  455438  2389   
                  1     0.10590205            Mara Rocha  455438  2389   
                  1     0.06111344               Petecão  455438  2389   
                  1     0.04688154                  Nulo  455438  2389   
                  1     0.02051067                Branco  455438  2389   
                  1     0.00669736         Marcio Bittar  455438  2389   
                  1     0.00125576      Professor Nilson  455438  2389   
                  1     0.00083717            David Hall  455438  2389   
           RO     1     0.35866388  Coronel Marcos Rocha  925763  2395   
                  1     0.34906054        Marcos Rogerio  925763  2395   
                  1     0.12025052            Léo Moraes  925763  2395   
                  1     0.08475992        Daniel Pereira  925763  2395   
                  1     0.05762004                  Nulo  925763  2395   
                  1     0.02797495                Branco  925763  2395   
                  1     0.00167015   Pimenta De Rondonia  925763  2395   
                  2     0.51064718  Coronel Marcos Rocha  925563  2395   
                  2     0.43966597        Marcos Rogério  925563  2395   
                  2     0.03131524                  Nulo  925563  2395   
                  2     0.01837161                Branco  925563  2395   
Senador    AC     1     0.33654249             Alan Rick  455438  2389   
                  1     0.15864378            Ney Amorim  455438  2389   
                  1     0.15027208    Dr. Jenilson Leite  455438  2389   
                  1     0.09208874         Nazaré Araújo  455438  2389   
                  1     0.08915864         Marcia Bittar  455438  2389   
                  1     0.06697363     Dra. Vanda Milani  455438  2389   
                  1     0.06069485                  Nulo  455438  2389   
                  1     0.03976559                Branco  455438  2389   
                  1     0.00418585       Sanderson Moura  455438  2389   
                  1     0.00167434          Dimas Sandas  455438  2389   
           RO     1     0.30939457       Jaime Bagattoli  925763  2395   
                  1     0.28726514      Mariana Carvalho  925763  2395   
                  1     0.10563674      Jaqueline Cassol  925763  2395   
                  1     0.09436326       Expedito Junior  925763  2395   
                  1     0.07766180                  Nulo  925763  2395   
                  1     0.06638831          Acir Gurgacz  925763  2395   
                  1     0.04801670                Branco  925763  2395   
                  1     0.00709812      Pastor Josinelio  925763  2395   
                  1     0.00417537  Dra Rosangela Lázaro  925763  2395   

                                 f          q      var_p       ep_p  \
cargo      estado turno                                               
Governador AC     1     0.00524550 0.46965257 0.00010376 0.01018613   
                  1     0.00524550 0.77354542 0.00007297 0.00854229   
                  1     0.00524550 0.89409795 0.00003944 0.00628037   
                  1     0.00524550 0.93888656 0.00002390 0.00488895   
                  1     0.00524550 0.95311846 0.00001861 0.00431435   
                  1     0.00524550 0.97948933 0.00000837 0.00289288   
                  1     0.00524550 0.99330264 0.00000277 0.00166469   
                  1     0.00524550 0.99874424 0.00000052 0.00072280   
                  1     0.00524550 0.99916283 0.00000035 0.00059029   
           RO     1     0.00258706 0.64133612 0.00009584 0.00978954   
                  1     0.00258706 0.65093946 0.00009467 0.00972963   
                  1     0.00258706 0.87974948 0.00004408 0.00663894   
    

### Presidente

In [58]:
proporcao_estrato = (
    amostra
    .query("cargo == 'Presidente'")
    .groupby(["cargo", "estado", "turno"])
    ["nome"]
    .apply(lambda s: s.value_counts(normalize=True))
    .to_frame("ph")
    .assign(
        nome=lambda df_: df_.index.get_level_values(-1)
    )
    .droplevel(-1)
    .join(sample_sizes)
    .assign(
        f=lambda df_: df_["n"] / df_["N"],
        qh=lambda df_: 1 - df_["ph"],
        var_ph=lambda df_: (1 - df_["f"]) * df_["ph"] * df_["qh"] / (df_["n"] - 1),
    )
)
proporcao_estrato

/var/folders/xs/y768pf612tscp94by_62zm040000gn/T/ipykernel_26301/3714606991.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["cargo", "estado", "turno"])


ph            nome       N    n          f  \
cargo      estado turno                                                      
Presidente AC     1     0.68181818  Jair Bolsonaro  455903  110 0.00024128   
                  1     0.25454545            Lula  455903  110 0.00024128   
                  1     0.02727273      Ciro Gomes  455903  110 0.00024128   
                  1     0.01818182          Branco  455903  110 0.00024128   
                  1     0.00909091    Simone Tebet  455903  110 0.00024128   
...                            ...             ...     ...  ...        ...   
           TO     1     0.01388889          Branco  891449  216 0.00024230   
                  2     0.52314815            Lula  871238  216 0.00024792   
                  2     0.43055556  Jair Bolsonaro  871238  216 0.00024792   
                  2     0.03703704            Nulo  871238  216 0.00024792   
                  2     0.00925926          Branco  871238  216 0.00024792   

                                qh     var_ph  
cargo      estado turno                        
Presidente AC     1     0.31818182 0.00198981  
                  1     0.74545455 0.00174042  
                  1     0.97272727 0.00024333  
                  1     0.98181818 0.00016373  
                  1     0.99090909 0.00008262  
...                            ...        ...  
           TO     1     0.98611111 0.00006369  
                  2     0.47685185 0.00116001  
                  2     0.56944444 0.00114008  
                  2     0.96296296 0.00016584  
                  2     0.99074074 0.00004266  

[75 rows x 7 columns]

In [59]:
peso_estrato = (
    sample_sizes
    .query("cargo == 'Presidente'")
    .groupby("turno", as_index=False)
    ["N"].apply(lambda s: s / s.sum())
    .to_frame("Wh")
    .droplevel(0)
)

In [60]:
(
    proporcao_estrato
    .join(peso_estrato)
    .assign(
        wh2varph=lambda df_: df_["Wh"].pow(2) * df_["var_ph"],
        whph=lambda df_: df_["Wh"] * df_["ph"],
    )
    .groupby(["nome", "turno"])
    .agg({
        "wh2varph": pd.Series.sum,
        "whph": pd.Series.sum
    })
    .rename(columns={
        "wh2varph": "var_p",
        "whph": "p"
    })
    .assign(
        ep_p =lambda df_: df_["var_p"].pow(.5),
        low_ci=lambda df_: df_["p"] - z * df_["ep_p"],
        upr_ci=lambda df_: df_["p"] + z * df_["ep_p"],
    )
)

var_p          p       ep_p      low_ci     upr_ci
nome             turno                                                        
Branco           1     0.00000412 0.00999090 0.00203065  0.00601090 0.01397090
                 2     0.00000463 0.01124825 0.00215208  0.00703024 0.01546625
Ciro Gomes       1     0.00000983 0.02414830 0.00313498  0.01800386 0.03029274
Felipe D'Avila   1     0.00000104 0.00249977 0.00101949  0.00050160 0.00449794
Jair Bolsonaro   1     0.00009825 0.44967990 0.00991213  0.43025249 0.46910732
                 2     0.00009887 0.48046300 0.00994335  0.46097438 0.49995162
Lula             1     0.00009879 0.45124266 0.00993956  0.43176148 0.47072383
                 2     0.00009884 0.48620420 0.00994162  0.46671899 0.50568942
Nulo             1     0.00000681 0.01664724 0.00260986  0.01153201 0.02176248
                 2     0.00000901 0.02208455 0.00300205  0.01620064 0.02796846
Padre Kelmon     1     0.00000052 0.00124769 0.00071980 -0.00016309 0.00265848
Simone Tebet     1     0.00001645 0.04121444 0.00405550  0.03326582 0.04916307
Soraya Thronicke 1     0.00000138 0.00332909 0.00117438  0.00102735 0.00563084

## Populacionais

### Governador e Senador

In [62]:
(
    populacao
    .query("cargo != 'Presidente'")
    .groupby(["cargo", "estado", "turno"])
    ["nome"].apply(lambda s: s.value_counts(normalize=True))
    .to_frame("P")
)

/var/folders/xs/y768pf612tscp94by_62zm040000gn/T/ipykernel_26301/954374765.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["cargo", "estado", "turno"])


P
cargo      estado turno                                  
Governador AC     1     Gladson Cameli         0.53157620
                        Jorge Viana            0.22673778
                        Mara Rocha             0.10357722
                        Petecão                0.06014650
                        Nulo                   0.04627853
...                                                   ...
Senador    TO     1     Pastor Claudemir Lopes 0.02350435
                        Andrea Schimdt         0.00659412
                        Lúcia Viana            0.00189190
                        Marcelo Claudio        0.00084534
                        Lazara Merley          0.00059579

[144 rows x 1 columns]

### Presidente

In [63]:
(
    populacao
    .query("cargo == 'Presidente'")
    .groupby(["cargo", "turno"])
    ["nome"]
    .apply(lambda s: s.value_counts(normalize=True))
    .to_frame("P")
    .unstack("turno")
)

/var/folders/xs/y768pf612tscp94by_62zm040000gn/T/ipykernel_26301/2066687084.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["cargo", "turno"])


P           
turno                                1          2
cargo                                            
Presidente Branco           0.00970590 0.00942752
           Ciro Gomes       0.02342258        NaN
           Eymael           0.00010700        NaN
           Felipe D'Avila   0.00181401        NaN
           Jair Bolsonaro   0.44291360 0.49438392
           Lula             0.45910471 0.47471680
           Léo Péricles     0.00019203        NaN
           Nulo             0.01541916 0.02147176
           Padre Kelmon     0.00061921        NaN
           Simone Tebet     0.04113684        NaN
           Sofia Manzano    0.00022125        NaN
           Soraya Thronicke 0.00515510        NaN
           Vera Lucia       0.00018860        NaN